<a href="https://colab.research.google.com/github/GiovaniValdrighi/NOTEARS/blob/master/notears_nonlinear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Notears não-linear
Implementação do algoritmo Notears nonlinear para o aprendizado de estruturas (DAG).

In [0]:
import tensorflow as tf
import numpy as np
import scipy.optimize as sopt
%load_ext tensorboard

In [0]:
def simulate_dag(d, s0, graph_type):
    """Simulate random DAG with some expected number of edges.
    Args:
        d (int): num of nodes
        s0 (int): expected num of edges
        graph_type (str): ER, SF, BP
    Returns:
        B (np.ndarray): [d, d] binary adj matrix of DAG
    """
    def _random_permutation(M):
        # np.random.permutation permutes first axis only
        P = np.random.permutation(np.eye(M.shape[0]))
        return P.T @ M @ P

    def _random_acyclic_orientation(B_und):
        return np.tril(_random_permutation(B_und), k=-1)

    def _graph_to_adjmat(G):
        return np.array(G.get_adjacency().data)

    if graph_type == 'ER':
        # Erdos-Renyi
        G_und = ig.Graph.Erdos_Renyi(n=d, m=s0)
        B_und = _graph_to_adjmat(G_und)
        B = _random_acyclic_orientation(B_und)
    elif graph_type == 'SF':
        # Scale-free, Barabasi-Albert
        G = ig.Graph.Barabasi(n=d, m=int(round(s0 / d)), directed=True)
        B = _graph_to_adjmat(G)
    elif graph_type == 'BP':
        # Bipartite, Sec 4.1 of (Gu, Fu, Zhou, 2018)
        top = int(0.2 * d)
        G = ig.Graph.Random_Bipartite(top, d - top, m=s0, directed=True, neimode=ig.OUT)
        B = _graph_to_adjmat(G)
    else:
        raise ValueError('unknown graph type')
    B_perm = _random_permutation(B)
    assert ig.Graph.Adjacency(B_perm.tolist()).is_dag()
    return B_perm

def simulate_nonlinear_sem(B, n, sem_type, noise_scale=None):
    """Simulate samples from nonlinear SEM.
    Args:
        B (np.ndarray): [d, d] binary adj matrix of DAG
        n (int): num of samples
        sem_type (str): mlp, mim, gp, gp-add
        noise_scale (np.ndarray): scale parameter of additive noise, default all ones
    Returns:
        X (np.ndarray): [n, d] sample matrix
    """
    def _simulate_single_equation(X, scale):
        """X: [n, num of parents], x: [n]"""
        z = np.random.normal(scale=scale, size=n)
        pa_size = X.shape[1]
        if pa_size == 0:
            return z
        if sem_type == 'mlp':
            hidden = 100
            W1 = np.random.uniform(low=0.5, high=2.0, size=[pa_size, hidden])
            W1[np.random.rand(*W1.shape) < 0.5] *= -1
            W2 = np.random.uniform(low=0.5, high=2.0, size=hidden)
            W2[np.random.rand(hidden) < 0.5] *= -1
            x = sigmoid(X @ W1) @ W2 + z
        elif sem_type == 'mim':
            w1 = np.random.uniform(low=0.5, high=2.0, size=pa_size)
            w1[np.random.rand(pa_size) < 0.5] *= -1
            w2 = np.random.uniform(low=0.5, high=2.0, size=pa_size)
            w2[np.random.rand(pa_size) < 0.5] *= -1
            w3 = np.random.uniform(low=0.5, high=2.0, size=pa_size)
            w3[np.random.rand(pa_size) < 0.5] *= -1
            x = np.tanh(X @ w1) + np.cos(X @ w2) + np.sin(X @ w3) + z
        elif sem_type == 'gp':
            from sklearn.gaussian_process import GaussianProcessRegressor
            gp = GaussianProcessRegressor()
            x = gp.sample_y(X, random_state=None).flatten() + z
        elif sem_type == 'gp-add':
            from sklearn.gaussian_process import GaussianProcessRegressor
            gp = GaussianProcessRegressor()
            x = sum([gp.sample_y(X[:, i, None], random_state=None).flatten()
                     for i in range(X.shape[1])]) + z
        else:
            raise ValueError('unknown sem type')
        return x

    d = B.shape[0]
    scale_vec = noise_scale if noise_scale else np.ones(d)
    X = np.zeros([n, d])
    G = ig.Graph.Adjacency(B.tolist())
    ordered_vertices = G.topological_sorting()
    assert len(ordered_vertices) == d
    for j in ordered_vertices:
        parents = G.neighbors(j, mode=ig.IN)
        X[:, j] = _simulate_single_equation(X[:, parents], scale_vec[j])
    return X


In [0]:
class Notears_MLP(tf.keras.models.Model):
  '''
  Class for the neural network used on NOTEARS non linear model
  
  Inputs:
    dims [int] - list of dimensions of hidden layers, the last dimension must be 1
    batch_size [int] - size of the batch for training
    bias [bool] - use of bias in model
  '''

  class bound_adj(tf.keras.constraints.Constraint):
    '''Class for fc1 weights constraints, the weights are non-negative and weights on diagonal are 0'''
    def __init__(self, n_variables, dims):
      self.n_variables = n_variables
      self.dims = dims
      return
    
    def __call__(self, w):
      w = w * tf.cast(tf.math.greater_equal(w, 0.), tf.float32)
      mask = tf.eye(self.n_variables, )
      for i in range(self.n_variables):
        for m in range(self.dims[1]):
          for j in range(self.n_variables):
            if i == j:
              pass
              #w[i + m + j] = 0. #não sei se é valido
      return w


  def __init__(self, dims, batch_size = 100, bias = True):
    super(Notears_MLP, self).__init__()
    self.dims = dims
    self.n_variables = dims[0]
    self.batch_size = batch_size
    #fc1 layer [d * m0]
    self.fc1_pos = tf.keras.layers.Dense(dims[0] * dims[1], input_shape = (batch_size, dims[0]), use_bias = bias)
    self.fc1_neg = tf.keras.layers.Dense(dims[0] * dims[1], input_shape = (batch_size, dims[0]), use_bias = bias)
    self.locally = []
    for i in range(len(dims) - 2):
      #fc2 layers [d, m1, m2]
      self.locally.append(tf.keras.layers.LocallyConnected1D(dims[i + 2], 1, input_shape = (batch_size, dims[0], dims[i + 1]), activation = 'sigmoid'))

  def layers_shape(self):
    '''Utility function for val_and_grad'''
    shapes = []
    for _ in range(2):
      shapes.append(self.fc1_pos.weights[0].shape)
      shapes.append(self.fc1_neg.weights[1].shape)
    for layer in self.locally:
      shapes.append(layer.weights[0].shape)
      shapes.append(layer.weights[1].shape)
    return shapes

  def call(self, inputs):
    '''
    Forward procedure in the neural network, pass the inputs trought fc1 and fc2 layers

    Inputs:
      inputs [tensor] - tensor of samples with shape [batch_size, n_variables]

    Outputs:
      out [tensor] - tensor with shape [batch_size, n_variables]
    '''

    hid = self.fc1_pos(inputs) - self.fc1_neg(inputs) #[n, d * m0]
    out = tf.reshape(hid, (self.batch_size, self.n_variables, -1)) #[n, d, m0]
    for layer in self.locally:
      out = layer(out)
    out = tf.squeeze(out, 2) #[n, d, 1] -> [n, d]
    return out

  def flat_params(self):
    '''
    Return flat vector of params to Scipy minimize
    Order is: fc1_pos wegiths bias - fc1_neg weights bias - layers weights bias
    '''
    params = []
    params.append(tf.reshape(self.fc1_pos.weights[0], tf.math.reduce_prod(self.fc1_pos.weights[0].shape)))
    params.append(self.fc1_pos.weights[1])
    params.append(tf.reshape(self.fc1_neg.weights[0], tf.math.reduce_prod(self.fc1_pos.weights[0].shape)))
    params.append(self.fc1_neg.weights[1])
    for layer in self.locally:
      params.append(tf.reshape(layer.weights[0], -1))
      params.append(tf.reshape(layer.weights[1], -1))
    return tf.concat(params, axis = 0).numpy().astype(np.float64)

  def flat_bounds(self):
    '''
    Return flat vector of bounds to Scipy minimize
    Order is: fc1_pos wegiths bias - fc1_neg weights bias - layers weights bias
    '''
    bounds = []
    bounds_fc1 = []
    for i in range(self.n_variables):
      for m in range(self.dims[1]):
        for j in range(self.n_variables):
          if i == j:
            bounds_fc1.append((0.,0.))
          else:
            bounds_fc1.append((0., None))
    bounds.append(bounds_fc1)
    bounds.append([(None, None) for _ in range(tf.math.reduce_prod(self.fc1_pos.weights[1].shape))])
    bounds.append(bounds_fc1)
    bounds.append([(None, None) for _ in range(tf.math.reduce_prod(self.fc1_pos.weights[1].shape))])
    for layer in self.locally:
      bounds.append([(None, None) for _ in range(tf.math.reduce_prod(layer.weights[0].shape))])
      bounds.append([(None, None) for _ in range(tf.math.reduce_prod(layer.weights[1].shape))])
    return sum(bounds, [])

  def update_params(self, weights):
    '''Function to update parameters from a flat params list'''

    shapes = self.layers_shape()
    s_flat = []
    for s in shapes:
      if len(s_flat) > 0:
        s_flat.append(np.prod(s) + s_flat[-1])
      else:
        s_flat.append(np.prod(s))
    self.fc1_pos.set_weights([np.reshape(weights[:s_flat[0]], self.fc1_pos.weights[0].shape),
                                       np.reshape(weights[s_flat[0]:s_flat[1]], self.fc1_pos.weights[1].shape)])
    self.fc1_neg.set_weights([np.reshape(weights[s_flat[1]:s_flat[2]], self.fc1_neg.weights[0].shape),
                                       np.reshape(weights[s_flat[2]:s_flat[3]], self.fc1_neg.weights[1].shape)])
    k = 3
    for layer in self.locally:
      layer.set_weights([np.reshape(weights[s_flat[k]:s_flat[k+1]], layer.weights[0].shape),
                                  np.reshape(weights[s_flat[k+1]:s_flat[k+2]], layer.weights[1].shape)])
      k+= 2


  def _h(self):
    '''Calculate the constraint of fc1 to ensure that it's a DAG'''
    fc1_weights = self.fc1_pos.weights[0] - self.fc1_neg.weights[0]
    fc1_weights = tf.reshape(fc1_weights, (self.n_variables, -1, self.n_variables))
    A = tf.transpose(tf.math.reduce_sum(fc1_weights, axis = 1))
    #(Yu et al. 2019 DAG-GNN)
    # h(w) = tr[(I + kA*A)^n_variables] - n_variables
    M = tf.eye(self.n_variables, num_columns = self.n_variables) + A/self.n_variables
    E = tf.pow(M, self.n_variables - 1)
    h = tf.math.reduce_sum(tf.transpose(E) * M) - self.n_variables
    return h
  
  def _l2_loss(self):
    '''Calculate L2 loss from model parameters'''
    loss = 0
    fc1_weights = self.fc1_pos.weights[0] - self.fc1_neg.weights[0]
    loss +=  tf.math.reduce_sum(tf.pow(fc1_weights, 2))
    for layer in self.locally:
      loss += tf.math.reduce_sum(tf.pow(layer.weights[0], 2))
    return loss

  def _l1_loss(self):
    '''Calculate L1 loss from fc1 parameters'''
    return tf.math.reduce_sum(self.fc1_pos.weights[0] + self.fc1_neg.weights[0])

  def to_adj(self):
    '''Reshape fc1 to an adjacency matrix'''
    fc1_weights = self.fc1_pos.weights[0] - self.fc1_neg.weights[0] #[d, d * m0]
    fc1_weights = tf.reshape(fc1_weights, (self.n_variables, -1, self.n_variables)) #[d, m0, d]
    return tf.transpose(tf.math.reduce_sum(fc1_weights, axis = 1)) #[d, d]


In [0]:
def notears_nonlinear(dims, X,  h_tol = 1e-4, threshold = 0.2, lambda1 = 0.5, lambda2 = 0.5, rho_max = 1e+20, max_iter = 1e+16):
  '''
    Function that apply the NOTEARS algorithm in a non linear model
    
    Args:
        dims (int) : list of dimensions for neural network
        X (numpy.matrix) : [n_samples, n_variables] samples matrix 
        h_tol (float) : tolerance for constraint, exit condition 
        threshold (float) : threshold for W_est edge values
        lambda1 (float) : L1 regularization parameter
        lambda2 (float) : L2 regularization parameter 
        rho_max (float) : max value for rho in augmented lagrangian
        max_iter (int) : max number of iterations
    Outputs:
        W_est (numpy.matrix): [n_variables, n_variables] estimated graph
    '''
  def square_loss(X, Y):
    '''Calculate mean square error from X Y'''
    n = X.shape[0]
    loss = 0.5 * tf.math.reduce_sum(tf.pow(X -Y, 2)) / n
    return tf.cast(loss, tf.float32)

  def val_and_grad(flat_params):
    '''Calculate loss value and gradient for Scipy optmize'''
    with tf.GradientTape() as tape:
      Y = model(X).numpy()
      mse_loss = square_loss(X, Y) 
      h = model._h()
      h_constraint = 0.5 * rho * h * h + alpha * h
      fc1_loss = lambda1 * model._l1_loss()
      locally_loss = 0.5 * lambda2 * model._l2_loss()
      loss = mse_loss + h_constraint + fc1_loss + locally_loss
    grad = tape.gradient(loss, model.trainable_variables)
    flat_grad = []
    i = 1
    for gradient in grad:
      if gradient != None:
        flat_grad.append(tf.reshape(gradient, -1).numpy())
      else:
        flat_grad.append(np.array([0 for _ in range(tf.math.reduce_prod(model.layers_shape()[i]))]))
        i+=2
    flat_grad = np.concatenate(flat_grad)
    return loss.numpy().astype(np.float64), flat_grad
    
  ########################
  # Optimization process #
  ########################
  model = Notears_MLP(dims, X.shape[0])
  _ = model(X)
  rho, alpha, h = 1., 0., np.inf
  for _ in range(int(max_iter)):
    h_new = None
    while rho < rho_max:
      sol = sopt.minimize(val_and_grad, model.flat_params(), method = "L-BFGS-B", jac = True, bounds = model.flat_bounds())
      new_params = sol.x
      model.update_params(new_params.astype(np.float32))
      h_new = model._h().numpy()

      #Updating rho constraint parameter
      if h_new > h * 0.25:
        rho = rho * 10
      else:
        break
    
    h = h_new

    #Ascent alpha
    alpha += rho * h

    #Verifying constraint tolerance
    if h < h_tol or rho >= rho_max:
      break

  #Applying threshold   
  W_est = model.to_adj().numpy()
  W_est[np.abs(W_est) < threshold] = 0
  return W_est  

In [0]:
dag_est = notears_nonlinear([5, 10, 1], X.astype(np.float32))

In [49]:
np.transpose(dag_est)

array([[ 0.        ,  0.        ,  0.31067073,  0.        , -0.53133166],
       [-0.65230167,  0.        ,  0.26083845,  0.        ,  0.        ],
       [ 0.5321901 ,  0.25866973,  0.        ,  0.22993842, -0.370006  ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-1.414167  ,  0.        ,  0.45628956,  0.        ,  0.        ]],
      dtype=float32)

In [89]:
G_true

array([[0., 0., 0., 0., 1.],
       [1., 0., 1., 1., 1.],
       [0., 0., 0., 1., 1.],
       [1., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0.]])

In [0]:
import numpy as np
from scipy.special import expit as sigmoid
import igraph as ig
import random
G_true = simulate_dag(5, 9, 'ER')
X = simulate_nonlinear_sem(G_true, 200, 'mim')



In [0]:
model = Notears_MLP([5, 10, 1])

In [11]:
out = model(X.astype(np.float32))
model.fc1_pos.weights[0].dtype

tf.float32

In [12]:
model.locally[0].weights[0].dtype

tf.float32

In [143]:
model.fc1_pos.weights

[<tf.Variable 'notears_mlp_28/dense_56/kernel:0' shape=(5, 50) dtype=float64, numpy=
 array([[-0.16499503, -0.30215633,  0.06343339,  0.07965161,  0.14066979,
         -0.13873502, -0.18235779, -0.124769  ,  0.30861387,  0.28699815,
          0.2386138 ,  0.11993983,  0.15438841,  0.192188  ,  0.05114096,
          0.05632323, -0.27395713, -0.0367157 , -0.30350054,  0.05087925,
         -0.31410413,  0.23411285,  0.17240236, -0.01798805,  0.31396076,
         -0.155281  ,  0.19146078,  0.05078613,  0.25499627,  0.18407879,
         -0.32438872,  0.03940999, -0.01468688,  0.26728438,  0.27723504,
          0.30798503, -0.28224893,  0.25932945, -0.05096385, -0.15652426,
         -0.02924489, -0.29804994,  0.27823251, -0.13527119, -0.05858691,
          0.1474097 , -0.16917319, -0.13944883, -0.24102308,  0.08237474],
        [-0.14934635,  0.06184217, -0.32488558, -0.08121349,  0.04778818,
          0.11236055, -0.01568407,  0.22147425,  0.03264974, -0.19912984,
         -0.01361658, -0.0

In [140]:
X.dtype

dtype('float64')